In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import json
import sys
from pathlib import Path

import ipywidgets as w
import ipyvuetify as v
import pdblp
import numpy as np
import pandas as pd
from IPython.display import display
from ipydatagrid import DataGrid
from sklearn.linear_model import LinearRegression

root_path = Path('/dev/projects/credit/backend/')

if (path := str(root_path.resolve())) not in sys.path:
    sys.path.append(path)
    
import utils
from api.gui.utils import tables
from api.gui.payoff import Form
from api.data.processing import get_current_data

In [6]:
data = get_current_data()

In [8]:
security = 'ZR767554 Corp'

In [12]:
periods = ['1w', '1m', '3m', '6m', 'ytd', '12m']
intervals = ['high', 'low', 'change']

In [16]:
field = 'z_spread'
fields = [field, *[f"{field}_{interval}_{period}" for interval in intervals for period in periods]]

In [173]:
data_ = data.loc[security, fields]
df_plot = pd.DataFrame(data_.index[1:].str.replace(f'{field}_', '').str.split('_').to_list(), columns=['interval', 'period'])

In [188]:
df_plot = pd.DataFrame(data_.index[1:].str.replace(f'{field}_', '').str.split('_').to_list(), columns=['interval', 'period'])
df_plot['value'] = data_[1:].to_numpy()
df_plot = pd.concat([pd.DataFrame([{'interval': 'now', 'period': '0d', 'value': data_[field]}]), df_plot]).reset_index(drop=True)

In [189]:
from plotly import graph_objects as go, express as px
from plotly.subplots import make_subplots
plotly_colors = px.colors.qualitative.Plotly

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

In [190]:
fig_ = px.scatter(
    df_plot.query(f'interval not in "change"'), 
    x='period', 
    y='value', 
    color='interval', 
    template='plotly_white',
    color_discrete_sequence=[plotly_colors[0], plotly_colors[2], plotly_colors[1]]
)

In [191]:
fig.add_traces(fig_.data)
fig.update_layout(fig_.layout)
fig.add_hline(y=df_plot.loc[0, 'value'], line_width=1, line_dash="dash", line_color=plotly_colors[0], opacity=.7);

In [192]:
df_change_plot = df_plot.query(f'interval in "change"')
df_change_plot = df_change_plot.assign(sign=np.sign(df_change_plot['value']))

In [193]:
fig_chg = px.bar(df_change_plot, x='period', y='value', color='sign')
fig_chg.update_traces(width=0.1, opacity=0.5);

In [194]:
fig.add_traces(
    fig_chg.data, 
    secondary_ys=[True for d in fig_chg.data]
)